In [ ]:
import timeit
import json
import ricci_regularization
import torch, yaml

In [ ]:
with open('../../experiments/MNIST_Setting_1_config.yaml', 'r') as yaml_file:
#with open('../../experiments/MNIST01_exp7_config.yaml', 'r') as yaml_file:
#with open('../../experiments/Swissroll_exp4_config.yaml', 'r') as yaml_file:
    yaml_config = yaml.load(yaml_file, Loader=yaml.FullLoader)

violent_saving = False # if False it will not save plots

d = 2

# Loading data and nn weights

In [ ]:
# Load data loaders based on YAML configuration
dict = ricci_regularization.DataLoaders.get_dataloaders(
    dataset_config=yaml_config["dataset"],
    data_loader_config=yaml_config["data_loader_settings"]
)
train_loader = dict["train_loader"]
test_loader = dict["test_loader"]
test_dataset = dict.get("test_dataset")  # Assuming 'test_dataset' is a key returned by get_dataloaders

print("Data loaders created successfully.")
additional_path="../"

In [ ]:
experiment_name = yaml_config["experiment"]["name"]

#Path_pictures = yaml_config["experiment"]["path"]
Path_pictures = additional_path + "../experiments/" + yaml_config["experiment"]["name"]
if violent_saving == True:
    # Check and create directories based on configuration
    if not os.path.exists(Path_pictures):  # Check if the picture path does not exist
        os.mkdir(Path_pictures)  # Create the directory for plots if not yet created
        print(f"Created directory: {Path_pictures}")  # Print directory creation feedback
    else:
        print(f"Directiry already exists: {Path_pictures}")

curv_w = yaml_config["loss_settings"]["lambda_curv"]

dataset_name = yaml_config["dataset"]["name"]
D = yaml_config["architecture"]["input_dim"]
# D is the dimension of the dataset
if dataset_name in ["MNIST01", "Synthetic"]:
    # k from the JSON configuration file is the number of classes
    #k = yaml_config["dataset"]["k"]
    k = len(yaml_config["dataset"]["selected_labels"])
    selected_labels = yaml_config["dataset"]["selected_labels"]
elif dataset_name == "MNIST":
    k = 10
print("Experiment name:", experiment_name)
print("Plots saved at:", Path_pictures)

In [ ]:
### Initialize the two networks

torus_ae, Path_ae_weights = ricci_regularization.DataLoaders.get_tuned_nn(config=yaml_config, additional_path = additional_path)

torus_ae = torus_ae.to("cpu")

print(f"AE weights loaded successfully from {Path_ae_weights}.")

encoder = torus_ae.encoder_torus
decoder = torus_ae.decoder_torus

# Timing

In [ ]:
# Define number of iterations for averaging
iterations = 100

# Define numsteps values
numsteps_values = [7, 15, 20, 30, 50,100]

timing_results = []

# Create a function to time both algorithms for different tgrid sizes

for numsteps in numsteps_values:
    # Build tgrid for the current numsteps
    tgrid = ricci_regularization.FiniteDifferences.make_grid(numsteps= numsteps)

    # Set up timeit environment dynamically for the current tgrid
    setup_code = f"""from __main__ import ricci_regularization, decoder, tgrid"""
    
    # Time for `Sc_fd`
    time_fd = timeit.timeit(
        stmt=f"ricci_regularization.Sc_fd(tgrid, function=decoder)",
        setup=setup_code,
        number=iterations
    )

    # Time for `Sc_jacfwd_vmap`
    time_jacfwd_vmap = timeit.timeit(
        stmt=f"ricci_regularization.Sc_jacfwd_vmap(tgrid, function=decoder)",
        setup=setup_code,
        number=iterations
    )

    # Save the results in a dictionary and append to the results list
    timing_results.append({
        'grid_size': f'{numsteps}x{numsteps}',
        'Sc_fd_avg_time': time_fd / iterations,
        'Sc_jacfwd_vmap_avg_time': time_jacfwd_vmap / iterations
    })

# Now `timing_results` contains the list of results
print("\nTiming Results:")
for result in timing_results:
    print(result)



# Save timing results to a JSON file
with open(Path_pictures+'/timing_results.json', mode='w') as file:
    json.dump(timing_results, file, indent=4)

print("Results saved to timing_results.json")

In [ ]:
import matplotlib.pyplot as plt

# Initialize lists to store the data
grid_sizes = []
sc_fd_times = []
sc_jacfwd_vmap_times = []

# Read the timing results from the JSON file
with open(Path_pictures+'/timing_results.json', 'r') as file:
    timing_results = json.load(file)
    for result in timing_results:
        grid_sizes.append(result['grid_size'])
        sc_fd_times.append(result['Sc_fd_avg_time'])
        sc_jacfwd_vmap_times.append(result['Sc_jacfwd_vmap_avg_time'])

# Plotting
plt.figure(figsize=(10, 6))

# Plot average times for Sc_fd and Sc_jacfwd_vmap
plt.plot(grid_sizes, sc_fd_times, marker='o', label='fd', linestyle='-')
plt.plot(grid_sizes, sc_jacfwd_vmap_times, marker='s', label='jacfwd', linestyle='-')

# Adding labels and title
plt.ylabel('Average Time (seconds)')
plt.xlabel('Grid Size')
plt.title('Performance Comparison of scalar curvature $R$ computation: fd vs jacfwd')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.grid()
plt.legend()

# Save the plot
plt.savefig(Path_pictures+'/timing_results_plot.pdf', bbox_inches='tight')
plt.show()